# Configuration

In [1]:
import trappy 

config = {}

# TRAPpy Events
config["THERMAL"] = trappy.thermal.Thermal
config["OUT"] = trappy.cpu_power.CpuOutPower
config["IN"] = trappy.cpu_power.CpuInPower
config["PID"] = trappy.pid_controller.PIDController
config["GOVERNOR"] = trappy.thermal.ThermalGovernor

# Control Temperature
config["CONTROL_TEMP"] = 77000

# A temperature margin of 2.5 degrees Celsius
config["TEMP_MARGIN"] = 2500

# The Sustainable power at the control Temperature
config["SUSTAINABLE_POWER"] = 2500

# Expected percentile of CONTROL_TEMP + TEMP_MARGIN
config["EXPECTED_TEMP_QRT"] = 95

# Maximum expected Standard Deviation as a percentage
# of mean temperature
config["EXPECTED_STD_PCT"] = 5


# Run Object

In [2]:
# Create a Trace object

TRACE = "/Users/kapileshwarsingh/AnalysisRawData/LPC/thermal"
run = trappy.Run(TRACE, "SomeBenchMark")

# Assertions

In [3]:
# Create an Assertion Object

from bart.common.Analyzer import Analyzer
t = Analyzer(run, config)

BIG = '000000f0'
LITTLE = '0000000f'

## Assertion: Load and Dynamic Power

<html>
This assertion makes sure that the dynamic power for the each cluster is zero when the sum of the "loads" of each CPU is 0

    $$\forall\ t\ |\ Load(t) = \sum\limits_{i=0}^{cpus} Load_i(t) = 0 \implies dynamic\ power(t)=0 $$
    
</html>

In [4]:
result = t.getStatement("((IN:load0 + IN:load1 + IN:load2 + IN:load3) == 0) \
                & (IN:dynamic_power > 0)",reference=True, select=BIG)
if len(result):
    print "FAIL: Dynamic Power is NOT Zero when load is Zero for the BIG cluster"
else:
    print "PASS: Dynamic Power is Zero when load is Zero for the BIG cluster"

    
result = t.getStatement("((IN:load0 + IN:load1 + IN:load2 + IN:load3) == 0) \
                & (IN:dynamic_power > 0)",reference=True, select=BIG)
if len(result):
    print "FAIL: Dynamic Power is NOT Zero when load is Zero for the LITTLE cluster"
else:
    print "PASS: Dynamic Power is Zero when load is Zero for the LITTLE cluster"

PASS: Dynamic Power is Zero when load is Zero for the BIG cluster
PASS: Dynamic Power is Zero when load is Zero for the LITTLE cluster


## Assertion: Control Temperature and Sustainable Power

<html>

When the temperature is greater than the control temperature, the total power granted to all cooling devices should be less than sustainable_power

$$\forall\ t\ |\ Temperature(t) > control\_temp \implies Total\ Granted\ Power(t) < sustainable\_power$$

<html/>

In [5]:
result = t.getStatement("(GOVERNOR:current_temperature > CONTROL_TEMP) &\
            (PID:output > SUSTAINABLE_POWER)", reference=True, select=0)

if len(result):
    print "FAIL: The Governor is allocating power > sustainable when T > CONTROL_TEMP"
else:
    print "PASS: The Governor is allocating power <= sustainable when T > CONTROL_TEMP"    

PASS: The Governor is allocating power <= sustainable when T > CONTROL_TEMP


# Statistics

Check if 95% of the temperature readings are below CONTROL_TEMP + MARGIN

In [6]:
t.assertStatement("numpy.percentile(THERMAL:temp, 95) < (CONTROL_TEMP + TEMP_MARGIN)")

True

Check if the mean temperauture is less than CONTROL_TEMP

In [7]:
t.assertStatement("numpy.mean(THERMAL:temp) <= CONTROL_TEMP",  select=0)

True

We can also use getStatement to get the absolute values. Here we are getting the standard deviation expresssed as a percentage of the mean

In [10]:
t.getStatement("(numpy.std(THERMAL:temp) * 100.0) / numpy.mean(THERMAL:temp)", select=0)

2.2390646863103232